In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
import pyarrow as pa

In [7]:
path = 'data'
obj = os.scandir(path)
l = [entry for entry in obj]
df_dict = {}

for entry in l:
    p = entry.name[0:-8]
    df_dict[p] = pd.read_parquet('data/'+entry.name, engine='pyarrow')


In [8]:
procedure = 'CPT58150'
df_dict[procedure].head().T

,0,1,2,3,4
ccn,290003,290039,290007,360144,100168
Procedure,CPT 58150,CPT 58150,CPT 58150,CPT 58150,CPT 58150
State Code,NV,NV,NV,OH,FL
Rural Versus Urban,U,U,U,U,U
PSI_03,0.13266,0.13266,0.13266,0.13266,0.13266
PSI_06,0.184709,0.184709,0.184709,0.184709,0.184709
PSI_08,0.114236,0.114236,0.114236,0.114236,0.114236
PSI_09,1.56662,1.56662,1.56662,1.56662,1.56662
PSI_10,0.650623,0.650623,0.650623,0.650623,0.650623
PSI_11,9.818181,9.818181,9.818181,9.818181,9.818181


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_dict[procedure].drop(columns='negotiated_rates_median'), 
                                                    df_dict[procedure]['negotiated_rates_median'], test_size=0.3, 
                                                    random_state=47)

In [11]:
names_list = ['ccn', 'Procedure', 'State Code','Rural Versus Urban']
names_train = X_train[names_list]
names_test = X_test[names_list]
X_train.drop(columns=names_list, inplace=True)
X_test.drop(columns=names_list, inplace=True)
X_train.shape, X_test.shape

((303, 34), (130, 34))

In [12]:
X_train.dtypes

PSI_03                                                                         float64
PSI_06                                                                         float64
PSI_08                                                                         float64
PSI_09                                                                         float64
PSI_10                                                                         float64
PSI_11                                                                         float64
PSI_12                                                                         float64
PSI_13                                                                         float64
PSI_14                                                                         float64
PSI_15                                                                         float64
PSI_90                                                                         float64
HAI_1_SIR                                  

In [13]:
X_test.dtypes

PSI_03                                                                         float64
PSI_06                                                                         float64
PSI_08                                                                         float64
PSI_09                                                                         float64
PSI_10                                                                         float64
PSI_11                                                                         float64
PSI_12                                                                         float64
PSI_13                                                                         float64
PSI_14                                                                         float64
PSI_15                                                                         float64
PSI_90                                                                         float64
HAI_1_SIR                                  

In [14]:
train_mean = y_train.mean()
train_mean

1341.9522442244224

In [15]:
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
dumb_reg.constant_

array([[1341.95224422]])

In [16]:
def r_squared(y, ypred):
    """R-squared score.
    
    Calculate the R-squared, or coefficient of determination, of the input.
    
    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    ybar = np.sum(y) / len(y) 
    sum_sq_tot = np.sum((y - ybar)**2) 
    sum_sq_res = np.sum((y - ypred)**2) 
    R2 = 1.0 - (sum_sq_res / sum_sq_tot)
    return R2

In [ ]:
y_tr_pred_ = train_mean * np.ones(len(y_train))
y_tr_pred_[:5]

In [ ]:
y_tr_pred = dumb_reg.predict(X_train)
y_tr_pred[:5]

In [ ]:
r_squared(y_train, y_tr_pred)

In [ ]:
y_te_pred = train_mean * np.ones(len(y_test))
r_squared(y_test, y_te_pred)

In [ ]:
def mae(y, ypred):
    """Mean absolute error.
    
    Calculate the mean absolute error of the arguments

    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    abs_error = np.abs(y - ypred)
    mae = np.mean(abs_error)
    return mae

In [ ]:
mae(y_train, y_tr_pred)

In [ ]:
mae(y_test, y_te_pred)

In [ ]:
def mse(y, ypred):
    """Mean square error.
    
    Calculate the mean square error of the arguments

    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    sq_error = (y - ypred)**2
    mse = np.mean(sq_error)
    return mse

In [ ]:
mse(y_train, y_tr_pred)

In [ ]:
mse(y_test, y_te_pred)

In [ ]:
np.sqrt([mse(y_train, y_tr_pred), mse(y_test, y_te_pred)])

In [ ]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

In [ ]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

In [ ]:
mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)

In [ ]:
r2_score(y_train, y_tr_pred), r2_score(y_tr_pred, y_train)

In [ ]:
r2_score(y_test, y_te_pred), r2_score(y_te_pred, y_test)

In [ ]:
r_squared(y_train, y_tr_pred), r_squared(y_tr_pred, y_train)

In [ ]:
r_squared(y_test, y_te_pred), r_squared(y_te_pred, y_test)

In [ ]:
X_defaults_median = X_train.median()
X_defaults_median

In [ ]:
X_tr = X_train.fillna(X_defaults_median)
X_te = X_test.fillna(X_defaults_median)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)
X_te_scaled = scaler.transform(X_te)

In [ ]:
lm = LinearRegression().fit(X_tr_scaled, y_train)

In [ ]:
y_tr_pred = lm.predict(X_tr_scaled)
y_te_pred = lm.predict(X_te_scaled)

In [ ]:
median_r2 = r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)
median_r2

In [ ]:
median_mae = mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)
median_mae

In [ ]:
median_mse = mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)
median_mse

In [ ]:
X_defaults_mean = X_train.mean()
X_defaults_mean

In [ ]:
X_tr = X_train.fillna(X_defaults_mean)
X_te = X_test.fillna(X_defaults_mean)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)
X_te_scaled = scaler.transform(X_te)

In [ ]:
lm = LinearRegression().fit(X_tr_scaled, y_train)

In [ ]:
y_tr_pred = lm.predict(X_tr_scaled)
y_te_pred = lm.predict(X_te_scaled)

In [ ]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

In [ ]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

In [ ]:
mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)

In [ ]:
pipe = make_pipeline(
    SimpleImputer(strategy='median'), 
    StandardScaler(), 
    LinearRegression()
)

In [ ]:
hasattr(pipe, 'fit'), hasattr(pipe, 'predict')

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_tr_pred = pipe.predict(X_train)
y_te_pred = pipe.predict(X_test)

In [ ]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

In [ ]:
median_r2

In [ ]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

In [ ]:
median_mae

In [ ]:
mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)

In [ ]:
median_mse

In [ ]:
pipe = make_pipeline(
    SimpleImputer(strategy='median'), 
    StandardScaler(),
    SelectKBest(score_func=f_regression),
    LinearRegression()
)

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_tr_pred = pipe.predict(X_train)
y_te_pred = pipe.predict(X_test)

In [ ]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

In [ ]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

In [ ]:
pipe15 = make_pipeline(
    SimpleImputer(strategy='median'), 
    StandardScaler(),
    SelectKBest(score_func=f_regression, k=15),
    LinearRegression()
)

In [ ]:
pipe15.fit(X_train, y_train)

In [ ]:
y_tr_pred = pipe15.predict(X_train)
y_te_pred = pipe15.predict(X_test)

In [ ]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

In [ ]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

In [ ]:
cv_results = cross_validate(pipe15, X_train, y_train, cv=5)

In [ ]:
cv_scores = cv_results['test_score']
cv_scores

In [ ]:
np.mean(cv_scores), np.std(cv_scores)

In [ ]:
np.round((np.mean(cv_scores) - 2 * np.std(cv_scores), np.mean(cv_scores) + 2 * np.std(cv_scores)), 2)

In [ ]:
pipe.get_params().keys()

In [ ]:
k = [k+1 for k in range(len(X_train.columns))]
grid_params = {'selectkbest__k': k}

In [ ]:
lr_grid_cv = GridSearchCV(pipe, param_grid=grid_params, cv=5, n_jobs=-1)

In [ ]:
lr_grid_cv.fit(X_train, y_train)

In [ ]:
score_mean = lr_grid_cv.cv_results_['mean_test_score']
score_std = lr_grid_cv.cv_results_['std_test_score']
cv_k = [k for k in lr_grid_cv.cv_results_['param_selectkbest__k']]

In [ ]:
lr_grid_cv.best_params_

In [ ]:
best_k = lr_grid_cv.best_params_['selectkbest__k']
plt.subplots(figsize=(10, 5))
plt.errorbar(cv_k, score_mean, yerr=score_std)
plt.axvline(x=best_k, c='r', ls='--', alpha=.5)
plt.xlabel('k')
plt.ylabel('CV score (r-squared)')
plt.title('Pipeline mean CV score (error bars +/- 1sd)');

In [ ]:
selected = lr_grid_cv.best_estimator_.named_steps.selectkbest.get_support()

In [ ]:
coefs = lr_grid_cv.best_estimator_.named_steps.linearregression.coef_
features = X_train.columns[selected]
pd.Series(coefs, index=features).sort_values(ascending=False)

In [ ]:
RF_pipe = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
    RandomForestRegressor(random_state=47)
)

In [ ]:
rf_default_cv_results = cross_validate(RF_pipe, X_train, y_train, cv=5)

In [ ]:
rf_cv_scores = rf_default_cv_results['test_score']
rf_cv_scores

In [ ]:
np.mean(rf_cv_scores), np.std(rf_cv_scores)

In [ ]:
n_est = [int(n) for n in np.logspace(start=1, stop=3, num=20)]
grid_params = {
        'randomforestregressor__n_estimators': n_est,
        'standardscaler': [StandardScaler(), None],
        'simpleimputer__strategy': ['mean', 'median']
}
grid_params

In [ ]:
rf_grid_cv = GridSearchCV(RF_pipe, param_grid=grid_params, cv=5, n_jobs=-1)

In [ ]:
rf_grid_cv.fit(X_train, y_train)

In [ ]:
rf_grid_cv.best_params_

In [ ]:
rf_best_cv_results = cross_validate(rf_grid_cv.best_estimator_, X_train, y_train, cv=5)
rf_best_scores = rf_best_cv_results['test_score']
rf_best_scores

In [ ]:
np.mean(rf_best_scores), np.std(rf_best_scores)

In [ ]:
plt.subplots(figsize=(10, 5))
imps = rf_grid_cv.best_estimator_.named_steps.randomforestregressor.feature_importances_
rf_feat_imps = pd.Series(imps, index=X_train.columns).sort_values(ascending=False)
rf_feat_imps.plot(kind='bar')
plt.xlabel('features')
plt.ylabel('importance')
plt.title('Best random forest regressor feature importances');

In [ ]:
lr_neg_mae = cross_validate(lr_grid_cv.best_estimator_, X_train, y_train, 
                            scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

In [ ]:
lr_mae_mean = np.mean(-1 * lr_neg_mae['test_score'])
lr_mae_std = np.std(-1 * lr_neg_mae['test_score'])
lr_mae_mean, lr_mae_std

In [ ]:
mean_absolute_error(y_test, lr_grid_cv.best_estimator_.predict(X_test))

In [ ]:
rf_neg_mae = cross_validate(rf_grid_cv.best_estimator_, X_train, y_train, 
                            scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

In [ ]:
rf_mae_mean = np.mean(-1 * rf_neg_mae['test_score'])
rf_mae_std = np.std(-1 * rf_neg_mae['test_score'])
rf_mae_mean, rf_mae_std

In [ ]:
mean_absolute_error(y_test, rf_grid_cv.best_estimator_.predict(X_test))

In [ ]:
fractions = [.2, .25, .3, .35, .4, .45, .5, .6, .75, .8, 1.0]
train_size, train_scores, test_scores = learning_curve(pipe, X_train, y_train, train_sizes=fractions)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

In [ ]:
plt.subplots(figsize=(10, 5))
plt.errorbar(train_size, test_scores_mean, yerr=test_scores_std)
plt.xlabel('Training set size')
plt.ylabel('CV scores')
plt.title('Cross-validation score as training set size increases');

In [ ]:
best_model = rf_grid_cv.best_estimator_
best_model.version = 1
best_model.pandas_version = pd.__version__
best_model.numpy_version = np.__version__
best_model.sklearn_version = sklearn_version
best_model.X_columns = [col for col in X_train.columns]
best_model.build_datetime = datetime.datetime.now()